# Fine-tuning SmolVLM on ChartLlama Dataset

This notebook demonstrates how to fine-tune the SmolVLM model on the ChartLlama dataset using parameter-efficient fine-tuning (LoRA).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/code')

In [ ]:
# Core dependencies
!pip install -U datasets
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U trl  # latest TRL library from HuggingFace
!pip install -q flash-attn --no-build-isolation
!pip install -q word2number

In [ ]:
# --- Standard Libraries ---
import os, sys, gc, time, json, re, math, string, traceback
from pathlib import Path
from functools import reduce
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union
from itertools import islice

# --- Data Science & Visualization ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from cycler import cycler
from tqdm.notebook import tqdm

# --- Image Processing ---
from PIL import Image

# --- Data Handling & Utils ---
from datasets import Dataset, load_dataset
from accelerate.utils import set_seed
from word2number import w2n

# --- Hugging Face Transformers ---
from transformers import (
    AutoProcessor, AutoConfig, Idefics3ForConditionalGeneration,
    BitsAndBytesConfig, TrainingArguments
)

# --- PEFT (Parameter-Efficient Fine-Tuning) ---
from peft import (
    LoraConfig, get_peft_model,
    prepare_model_for_kbit_training, PeftModel
)

# --- TRL (Supervised Fine-Tuning) ---
from trl import SFTTrainer, SFTConfig

# --- Experiment Tracking & Deep Learning ---
import wandb
import torch


In [ ]:
print("--- Setting Configuration --- ")

# --- Basic Setup ---
WORKING_DIR = '/content/drive/MyDrive/code'
DATA_DIR = Path("./chartllama_data")
OUTPUT_DIR_BASE_NAME = "smolvlm-chartllama-sft-final" # change to short prompt for short prompt
# OUTPUT_DIR_BASE_NAME = "smolvlm-chartllama-sft-refactored"
SEED = 42
set_seed(SEED)

In [ ]:
if os.getcwd() != WORKING_DIR:
    print(f"Changing working directory to: {WORKING_DIR}")
    os.makedirs(WORKING_DIR, exist_ok=True)
    os.chdir(WORKING_DIR)
else:
    print(f"Already in working directory: {WORKING_DIR}")

In [ ]:
MODEL_ID = "HuggingFaceTB/SmolVLM-256M-Instruct" # specify model to use here

USE_LORA = True
USE_QLORA = True

LORA_R = 8 # rank set to 8
LORA_ALPHA = 16 # default of 8
LORA_DROPOUT = 0.1
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj",  # attention heads
                       "gate_proj", "up_proj", "down_proj", "lm_head"
                       ]

In [ ]:
if USE_QLORA and not USE_LORA:
    print("Warning: USE_QLORA=True requires USE_LORA=True. Forcing USE_LORA=True.")
    USE_LORA = True

In [ ]:
# --- Training Hyperparameters ---
NUM_TRAIN_EPOCHS = 6 # keep to default of 3
PER_DEVICE_TRAIN_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 1e-3
MAX_SEQ_LENGTH = 2048
OPTIMIZER = "adamw_torch_fused"
LR_SCHEDULER_TYPE = "cosine" # default to cosine
LOGGING_STEPS = 25
SAVE_STRATEGY = "epoch"
EVAL_STRATEGY = "epoch"
EVAL_STEPS = 50
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01 # default set to 0.01
SAMPLE_LIMIT = None # training sample limit on Chartllama, None will use everything available.

In [ ]:
# --- Resources & Precision ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16
ATTN_IMPLEMENTATION = "flash_attention_2"

In [ ]:
# --- Output Directory (Directly on Drive) ---
if not USE_LORA:
    TRAINING_TYPE = "full-tuned"
elif USE_QLORA:
    TRAINING_TYPE = "qlora-tuned"
else:
    TRAINING_TYPE = "lora-tuned"

In [ ]:
model_name_short = MODEL_ID.split('/')[-1]
output_folder_name = f"{OUTPUT_DIR_BASE_NAME}-{model_name_short}-{TRAINING_TYPE}-lalpha16-lr1e-3-6epochs" # test for lora alpha 16
OUTPUT_DIR = os.path.join(WORKING_DIR, output_folder_name) # full path on Drive
FINAL_MODEL_DIR = os.path.join(OUTPUT_DIR, "final_model")
FINAL_PROCESSOR_DIR = os.path.join(OUTPUT_DIR, "final_processor") # sub directory to save processor

In [ ]:
# --- Evaluation Config ---
CHARTQA_DATASET_ID = "HuggingFaceM4/ChartQA"
EVAL_SPLIT = "test"
EVAL_LIMIT = None # number of chartqa samples to use when evaluating ChartQA
MAX_NEW_TOKENS_EVAL = 32
EVAL_OUTPUT_FILE = os.path.join(OUTPUT_DIR, "chartqa_evaluation_results_comparison.json")

In [ ]:
# --- W&B Logging ---
WANDB_PROJECT = "smolvlm-chartllama-sft-refactored" # Set to None to disable
WANDB_RUN_NAME = f"{model_name_short}-{TRAINING_TYPE}-{int(time.time())}" # Unique name for run

In [ ]:
# Create output directory now (important for direct-to-drive)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory ensured: '{OUTPUT_DIR}'")

# display summary of all relevant information based on above settings.
# --- Print Summary ---
print(f"\n--- Configuration Summary ---")
print(f"Model ID: {MODEL_ID}")
print(f"Training Type: {TRAINING_TYPE}")
print(f"Use LoRA: {USE_LORA}, Use QLoRA: {USE_QLORA}")
print(f"Device: {DEVICE}, Compute Dtype: {DTYPE}, Attention: {ATTN_IMPLEMENTATION}")
print(f"Learning Rate: {LEARNING_RATE}, Epochs: {NUM_TRAIN_EPOCHS}")
print(f"Eval Strategy: {EVAL_STRATEGY}")
print(f"Output Directory (on Drive): '{OUTPUT_DIR}'")
print(f"Final Model Save Directory: '{FINAL_MODEL_DIR}'")
print(f"Final Processor Save Directory: '{FINAL_PROCESSOR_DIR}'")
print(f"Evaluation results file: {EVAL_OUTPUT_FILE}")
print(f"WandB Project: {WANDB_PROJECT}, Run Name: {WANDB_RUN_NAME}")
print("-----------------------------")

print("\n--- Full Configuration Complete --- ")

print("\n--- Configuration Complete --- ")

In [ ]:
# api key: 0469802d14d997b8dad4d23a7ba212e0a8d8f197
# --- Simplified WandB setup ---
print("--- WandB Setup ---")

if WANDB_PROJECT:
    try:
        wandb.login(relogin=False)
        wandb.init(
            project=WANDB_PROJECT,
            name=WANDB_RUN_NAME,
            config={
                "model_id": MODEL_ID,
                "training_type": TRAINING_TYPE,
                "use_lora": USE_LORA,
                "use_qlora": USE_QLORA,
                "dtype": str(DTYPE),
                "seed": SEED,
                "lora_r": LORA_R if USE_LORA else None,
                "lora_alpha": LORA_ALPHA if USE_LORA else None,
                "num_epochs": NUM_TRAIN_EPOCHS,
                "learning_rate": LEARNING_RATE,
                "batch_size": PER_DEVICE_TRAIN_BATCH_SIZE,
                "grad_accum": GRADIENT_ACCUMULATION_STEPS,
                "max_seq_length": MAX_SEQ_LENGTH,
                "output_dir": OUTPUT_DIR,
                "optimizer": OPTIMIZER,
                "lr_scheduler": LR_SCHEDULER_TYPE,
                "warmup_ratio": WARMUP_RATIO,
                "weight_decay": WEIGHT_DECAY,
            },
            job_type="fine-tuning"
        )
        print("WandB initialized.")
    except Exception as e:
        print(f"WandB initialization failed: {e}. Disabling WandB.")
        os.environ["WANDB_DISABLED"] = "true"
else:
    print("WandB disabled by configuration (WANDB_PROJECT=None).")
    os.environ["WANDB_DISABLED"] = "true"

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("--- WandB Setup Complete ---")

In [ ]:
# --- Define chartllama data loading function ---

print("\n--- Defining Data Loading Function ---")

def load_chartllama_data(data_root: Path, image_root: Path, limit: Optional[int] = None):
    """Loads ChartLlama QA pairs from JSON files."""
    if not data_root.is_dir():
        raise FileNotFoundError(f"Data root directory not found: {data_root.resolve()}")

    examples, skipped = [], 0
    json_files = list(data_root.glob("*_simplified_qa.json"))

    if not json_files:
        raise FileNotFoundError(f"No '*_simplified_qa.json' files found in {data_root}")

    for json_file in tqdm(json_files, desc="Processing JSON files"):
        with open(json_file, "r", encoding="utf-8") as f:
            qa_list = json.load(f)

        for idx, qa in enumerate(qa_list):
            if limit and len(examples) >= limit:
                break

            question = qa.get("conversations", [{}])[0].get("value", "").replace("<image>", "").strip()
            answer = qa.get("conversations", [{}, {}])[1].get("value", "").strip()
            image_path = (image_root / qa.get("image", "")).resolve()

            if question and answer and image_path.is_file():
                examples.append({
                    "id": qa.get("id", f"{json_file.stem}_{idx}"),
                    "question": question,
                    "answer": answer,
                    "image_path": str(image_path),
                })
            else:
                skipped += 1

    if not examples:
        raise ValueError(f"No valid examples loaded from {data_root}. Skipped {skipped} samples.")

    print(f"\nLoaded {len(examples)} examples. Skipped {skipped} invalid samples.")
    return Dataset.from_list(examples)

print("Data loading function defined.")

In [ ]:
# --- Load Chartllama data ---

print("\n--- Loading and Splitting Data ---")
train_dataset, eval_dataset, raw_dataset = None, None, None

# ensure DATA_DIR is a valid Path object
DATA_DIR = Path(DATA_DIR)
if not DATA_DIR.is_absolute():
    DATA_DIR = Path(WORKING_DIR) / DATA_DIR

if not DATA_DIR.is_dir():
    raise FileNotFoundError(f"Input data directory not found: {DATA_DIR.resolve()}")

print(f"Loading data from: {DATA_DIR.resolve()}")
raw_dataset = load_chartllama_data(DATA_DIR, DATA_DIR, limit=SAMPLE_LIMIT)

if not raw_dataset:
    raise RuntimeError("Data loading returned an empty dataset.")

# split dataset: 80% train, 20% validation
dataset_split = raw_dataset.train_test_split(test_size=0.2, seed=SEED, shuffle=True)
train_dataset, eval_dataset = dataset_split["train"], dataset_split["test"]

print(f"\nRaw data loaded and split.")
print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples (Validation): {len(eval_dataset)}")

# display one sample from train dataset
if train_dataset:
    print("\nSample train data point (raw):")
    print(train_dataset[0])
else:
    print("Train dataset is empty after split.")

In [ ]:
# ### 3.1 Load Model and Processor

print("\n--- Loading Model and Processor for Training ---")

# load Processor
print(f"Loading processor for model: {MODEL_ID}")
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
if processor.tokenizer.pad_token_id is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token
    if hasattr(processor, 'pad_token') and processor.pad_token is None:
        processor.pad_token = processor.tokenizer.eos_token
print("Processor loaded.")

# configure Model Loading
print("\nConfiguring model loading...")
model_load_kwargs = {"device_map": "auto", "torch_dtype": DTYPE}

if USE_LORA and USE_QLORA:
    print("Configuring model for QLoRA (4-bit).")
    model_load_kwargs["quantization_config"] = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=DTYPE
    )
elif USE_LORA:
    print(f"Configuring model for LoRA (dtype: {DTYPE}).")
else:
    print(f"Configuring model for Full Fine-Tuning (dtype: {DTYPE}).")

# load Config
config = AutoConfig.from_pretrained(MODEL_ID, trust_remote_code=True, use_cache=False)
if hasattr(config, "attn_implementation"):
    config.attn_implementation = ATTN_IMPLEMENTATION

# load Model
print(f"Loading model '{MODEL_ID}'...")
model = Idefics3ForConditionalGeneration.from_pretrained(
    MODEL_ID, config=config, trust_remote_code=True, **model_load_kwargs
)

print("Model loaded.")
print(f"  - Device Map: {getattr(model, 'hf_device_map', model.device)}")
print(f"  - Attention Implementation: {getattr(model.config, 'attn_implementation', 'N/A')}")

# PEFT Setup
if USE_LORA:
    print("\nApplying PEFT/LoRA configuration...")
    if USE_QLORA:
        model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True, gradient_checkpointing_kwargs={"use_reentrant": False})
    elif hasattr(model, "gradient_checkpointing_enable"):
        model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
        model.config.use_cache = False

    peft_config = LoraConfig(
        r=LORA_R, lora_alpha=LORA_ALPHA, lora_dropout=LORA_DROPOUT,
        target_modules=LORA_TARGET_MODULES, bias="none", task_type="CAUSAL_LM"
    )

    if not isinstance(model, PeftModel):
        model = get_peft_model(model, peft_config)
        model.print_trainable_parameters()
else:
    if hasattr(model, "gradient_checkpointing_enable"):
        model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
        model.config.use_cache = False

print("\n--- Model and Processor Setup Complete ---")

In [ ]:
# ### 3.1 define smolvlm formatting functions and collate_fn

print("\n--- Defining Formatting Function & Data Collator ---")

SYSTEM_MESSAGE = """You are a Vision Language Model specialized in interpreting visual data from chart images.
Your task is to analyze the provided chart image and respond to queries with concise answers, usually a single word, number, or short phrase.
The charts include a variety of types (e.g., line charts, bar charts) and contain colors, labels, and text.
Focus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary."""

# SYSTEM_MESSAGE = """You are a Vision-Language Model specialized in interpreting chart images. Please answer the question using only one word or a number based on the provided chart."""

def create_chat_messages(sample):
    """Creates the chat message structure for SFT training."""
    question = sample.get("question")
    answer = sample.get("answer")
    if question is None or answer is None:
         return None

    return [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_MESSAGE}]},
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": question}
        ]},
        {"role": "assistant", "content": [{"type": "text", "text": answer}]},
    ]


@dataclass # ChartLlama collator class for collate function
class ChartLlamaCollator:
    processor: AutoProcessor
    max_seq_length: int = MAX_SEQ_LENGTH

    def __post_init__(self):
        image_token_str = "<image>"
        self.image_token_id = self.processor.tokenizer.convert_tokens_to_ids(image_token_str)
        if self.image_token_id is None:
            self.image_token_id = -100

    def __call__(self, examples: List[Dict[str, Any]]) -> Dict[str, Any]:
        pil_images, texts = [], []

        for ex in examples:
            img_path = ex.get('image_path')
            if not img_path or not os.path.exists(img_path):
                continue

            img = Image.open(img_path).convert("RGB")
            messages = create_chat_messages(ex)
            if messages is None:
                continue

            formatted_text = self.processor.apply_chat_template(
                messages, add_generation_prompt=False, tokenize=False
            )

            pil_images.append(img)
            texts.append(formatted_text)

        if not texts or not pil_images:
            return {"input_ids": torch.tensor([], dtype=torch.long)}

        batch = self.processor(
            text=texts,
            images=pil_images,
            return_tensors="pt",
            padding=True,
            # truncation=True,
            # max_length=self.max_seq_length,
        )

        labels = batch["input_ids"].clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        if self.image_token_id != -100:
            labels[labels == self.image_token_id] = -100

        batch["labels"] = labels
        return batch

chart_collator = ChartLlamaCollator(processor=processor)
print("\nCustom Collator instantiated.")

In [ ]:
# configure training arguments
print("\n--- Configuring Training Arguments ---")

if 'OUTPUT_DIR' not in locals():
    raise NameError("OUTPUT_DIR not defined. Rerun configuration cell.")

trainer_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="",
    packing=False,
    dataset_kwargs={"skip_prepare_dataset": True},

    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},

    optim=OPTIMIZER,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    lr_scheduler_type=LR_SCHEDULER_TYPE,

    logging_strategy="steps",
    logging_steps=LOGGING_STEPS,
    save_strategy=SAVE_STRATEGY,
    save_total_limit=2,
    save_only_model=("peft_config" in locals() and peft_config is not None),

    bf16=(DTYPE == torch.bfloat16),
    fp16=(DTYPE == torch.float16),

    eval_strategy=EVAL_STRATEGY,
    eval_steps=EVAL_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    seed=SEED,
    report_to="wandb" if WANDB_PROJECT and os.environ.get("WANDB_DISABLED") != "true" else "none",
    remove_unused_columns=False,
)

print("SFTConfig set.")

In [ ]:
# initialize trainer

print("\n--- Initializing SFTTrainer ---")

required_components = [train_dataset, model, trainer_config, chart_collator, processor]

if all(required_components):
    trainer = SFTTrainer(
        model=model,
        args=trainer_config,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset if EVAL_STRATEGY != "no" else None,
        data_collator=chart_collator,
        peft_config=peft_config if USE_LORA else None,
    )
    print("SFTTrainer initialized successfully.")
else:
    missing = [name for name, comp in zip(
        ['train_dataset', 'model', 'trainer_config', 'chart_collator', 'processor'],
        required_components) if comp is None]
    raise RuntimeError(f"Cannot initialize trainer. Missing components: {', '.join(missing)}")

In [ ]:
# --- Run Training (with Loss curve Plotting Added & Enhanced Naming) ---

print("\n--- Checking if Training Should Be Skipped ---")

model_exists = os.path.isdir(FINAL_MODEL_DIR) and any(
    f.endswith(('.bin', '.safetensors', 'adapter_config.json')) for f in os.listdir(FINAL_MODEL_DIR)
)

processor_exists = os.path.isfile(os.path.join(FINAL_PROCESSOR_DIR, "preprocessor_config.json"))

if model_exists and processor_exists:
    print("Model and processor already exist. Skipping training.")
else:
    if trainer is None or processor is None:
        raise RuntimeError("Trainer or processor not available. Cannot proceed with training.")

    print("Starting training...")
    start_time = time.time()

    os.makedirs(FINAL_MODEL_DIR, exist_ok=True)
    os.makedirs(FINAL_PROCESSOR_DIR, exist_ok=True)

    train_result = trainer.train()

    print(f"Training completed in {(time.time() - start_time)/60:.2f} minutes.")

    trainer.save_model(FINAL_MODEL_DIR)
    processor.save_pretrained(FINAL_PROCESSOR_DIR)

    trainer.log_metrics("train", train_result.metrics)
    trainer.save_metrics("train", train_result.metrics)
    trainer.save_state()

    log_history = trainer.state.log_history
    if log_history:
        log_df = pd.DataFrame(log_history)
        fig, ax = plt.subplots(figsize=(12, 7))

        if 'loss' in log_df:
            ax.plot(log_df['step'], log_df['loss'], label='Training Loss', marker='o')
        if 'eval_loss' in log_df:
            ax.plot(log_df['step'], log_df['eval_loss'], label='Validation Loss', marker='s')

        ax.set_title(f"Training & Validation Loss ({model_name_short}, {TRAINING_TYPE})")
        ax.set_xlabel("Steps")
        ax.set_ylabel("Loss")
        ax.legend()
        ax.grid(True)

        plot_path = os.path.join(OUTPUT_DIR, f"loss_curves_{model_name_short}_{TRAINING_TYPE}.png")
        fig.savefig(plot_path, dpi=150)
        plt.close(fig)

# Cleanup
del trainer
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n--- Training Step Execution Finished ---")

In [ ]:
# --- Load Fine-tuned Model ---
print("\n--- Setting up Evaluation Environment ---")

FT_MODEL_LOAD_PATH = FINAL_MODEL_DIR
PROCESSOR_LOAD_PATH = FINAL_PROCESSOR_DIR

print(f"Loading processor from: {PROCESSOR_LOAD_PATH}")

if not os.path.isdir(PROCESSOR_LOAD_PATH):
    raise FileNotFoundError(f"Processor directory not found: {PROCESSOR_LOAD_PATH}")

eval_processor = AutoProcessor.from_pretrained(PROCESSOR_LOAD_PATH, trust_remote_code=True)

if eval_processor.tokenizer.pad_token_id is None:
    eval_processor.tokenizer.pad_token = eval_processor.tokenizer.eos_token
    if hasattr(eval_processor, 'pad_token') and eval_processor.pad_token is None:
        eval_processor.pad_token = eval_processor.tokenizer.eos_token

print("Processor loaded successfully.")

In [ ]:
# --- Load Fine-tuned Model ---

eval_model = None
if not eval_processor:
    raise RuntimeError("Processor is not loaded. Cannot load model.")

if not os.path.isdir(FT_MODEL_LOAD_PATH):
    raise FileNotFoundError(f"Fine-tuned model directory not found: {FT_MODEL_LOAD_PATH}")

# Load evaluation configuration
try:
    eval_config = AutoConfig.from_pretrained(FT_MODEL_LOAD_PATH, trust_remote_code=True)
except OSError:
    eval_config = AutoConfig.from_pretrained(MODEL_ID, trust_remote_code=True)

eval_config.use_cache = True
if hasattr(eval_config, "attn_implementation"):
    eval_config.attn_implementation = ATTN_IMPLEMENTATION

load_kwargs = {"device_map": "auto", "torch_dtype": DTYPE}

if USE_LORA:
    # Ensure pad_token consistency
    pad_token_id = eval_processor.tokenizer.pad_token_id
    base_config = AutoConfig.from_pretrained(MODEL_ID, trust_remote_code=True)

    config_to_modify = base_config.text_config if hasattr(base_config, 'text_config') else base_config
    config_to_modify.pad_token_id = pad_token_id
    config_to_modify.vocab_size = max(config_to_modify.vocab_size, pad_token_id + 1)

    base_config.use_cache = True
    if hasattr(base_config, "attn_implementation"):
        base_config.attn_implementation = ATTN_IMPLEMENTATION

    # if USE_QLORA: # do not use quantization
    #     load_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=DTYPE)

    base_model = Idefics3ForConditionalGeneration.from_pretrained(MODEL_ID, config=base_config, trust_remote_code=True, **load_kwargs)
    eval_model = PeftModel.from_pretrained(base_model, FT_MODEL_LOAD_PATH)
else:
    eval_model = Idefics3ForConditionalGeneration.from_pretrained(FT_MODEL_LOAD_PATH, config=eval_config, trust_remote_code=True, **load_kwargs)

# Prepare model for evaluation
eval_model.eval()
print("Fine-tuned model loaded successfully.")

In [ ]:
# --- Load Base Model for Evaluation ---

if eval_processor is None:
    raise RuntimeError("Processor is not loaded; cannot load base model.")

base_model_config = AutoConfig.from_pretrained(MODEL_ID, trust_remote_code=True)
base_model_config.use_cache = True

if hasattr(base_model_config, "attn_implementation"):
    base_model_config.attn_implementation = ATTN_IMPLEMENTATION

base_load_kwargs = {"device_map": "auto", "torch_dtype": DTYPE}

base_model_eval = Idefics3ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    config=base_model_config,
    trust_remote_code=True,
    **base_load_kwargs
)

base_model_eval.eval()
print("Base model loaded successfully.")

In [ ]:
# --- Load Evaluation Dataset (ChartQA) ---
chartqa_test_dataset = None
if eval_processor:
    try:
        print(f"\nLoading evaluation dataset '{CHARTQA_DATASET_ID}' split '{EVAL_SPLIT}'...")
        chartqa_test_iterable = load_dataset(CHARTQA_DATASET_ID, split=EVAL_SPLIT, streaming=False)

        if EVAL_LIMIT is not None and EVAL_LIMIT > 0:
            chartqa_test_list = list(chartqa_test_iterable)
            chartqa_test_dataset = chartqa_test_list[:EVAL_LIMIT]
            print(f"Loaded and limited to {len(chartqa_test_dataset)} samples for evaluation.")
        else:
            chartqa_test_dataset = list(chartqa_test_iterable)
            print(f"Loaded {len(chartqa_test_dataset)} samples for evaluation (full {EVAL_SPLIT} set).")

        if chartqa_test_dataset:
             if isinstance(chartqa_test_dataset[0], dict) and 'img_idx' not in chartqa_test_dataset[0]:
                  chartqa_test_dataset = [dict(sample, img_idx=i) for i, sample in enumerate(chartqa_test_dataset)]
        else:
             print("Warning: ChartQA dataset is empty after loading/limiting.")

    except Exception as e_dataset:
        print(f"ERROR loading evaluation dataset: {e_dataset}")
        traceback.print_exc()
        chartqa_test_dataset = None
else:
     print("Skipping dataset loading due to processor error.")

In [ ]:
# Check Evaluation Readiness
components = {
    "Processor": eval_processor,
    "Fine-tuned Model": eval_model,
    "Base Model": base_model_eval,
    "Dataset": chartqa_test_dataset if chartqa_test_dataset else None,
}

readiness = {name: comp is not None for name, comp in components.items()}
readiness["Dataset"] = readiness["Dataset"] and len(chartqa_test_dataset) > 0

can_evaluate_finetuned = all([readiness["Processor"], readiness["Fine-tuned Model"], readiness["Dataset"]])
can_evaluate_base = all([readiness["Processor"], readiness["Base Model"], readiness["Dataset"]])

print("\nEvaluation Readiness:")
for name, ready in readiness.items():
    print(f"  - {name} Ready: {ready}")

print(f"  - Can Evaluate Fine-tuned: {can_evaluate_finetuned}")
print(f"  - Can Evaluate Base: {can_evaluate_base}")
print(f"Evaluation Device: {DEVICE}")

# cleanup
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n--- Evaluation Setup Complete ---")

In [ ]:
# Define Evaluation helper functions
print("\n--- Defining Evaluation Helper Functions ---")

# System message for evaluation prompting (provided by SmolVLM)
EVAL_SYSTEM_MESSAGE = """You are a Vision Language Model specialized in interpreting visual data from chart images.
Your task is to analyze the provided chart image and respond to queries with concise answers, usually a single word, number, or short phrase.
The charts include a variety of types (e.g., line charts, bar charts) and contain colors, labels, and text.
Focus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary."""

# EVAL_SYSTEM_MESSAGE = """Have fun!"""

def create_inference_chat_messages(sample):
    """Creates the chat message structure for inference using ChartQA's 'query' field."""
    question = sample.get("query")
    if not question:
        return None
    return [
        {"role": "system", "content": [{"type": "text", "text": EVAL_SYSTEM_MESSAGE}]},
        {"role": "user", "content": [{"type": "image"}, {"type": "text", "text": question}]},
    ]


print("Evaluation helper functions defined.")

In [ ]:
# --- Generate Predictions and Save Results ---

print("\n--- Running Evaluation Generation & Saving ---")

# --- Define Concise Generation Function ---
def generate_prediction_concise(model, processor, sample, max_tokens, max_len):
    question, image = sample.get("query"), sample.get("image")
    if not (question and isinstance(image, Image.Image)): return "ERROR:Input"
    if image.mode != 'RGB':
        try: image = image.convert('RGB')
        except: return "ERROR:Convert"

    messages = create_inference_chat_messages(sample)
    if not messages: return "ERROR:Messages"

    prediction = "ERROR:Generate"
    inputs = None; generated_ids = None
    try:
        prompt = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
        device = next(iter(model.parameters()), torch.tensor([])).device
        if device is None: device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        inputs = processor(text=[prompt], images=[image], return_tensors="pt", padding=False, truncation=True, max_length=max_len).to(device)
        generated_ids = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=False,
                                       pad_token_id=processor.tokenizer.pad_token_id,
                                       eos_token_id=processor.tokenizer.eos_token_id)
        gen_tokens = generated_ids[0, inputs["input_ids"].shape[1]:]
        prediction = processor.decode(gen_tokens, skip_special_tokens=True).strip() if gen_tokens.numel() > 0 else "[NO_TOKENS]"
    except Exception as e:
        prediction = f"ERROR:{e.__class__.__name__}"
    finally:
        del inputs, generated_ids
    return prediction
# --- End Generation Function ---

# Determine models to evaluate
models_to_eval = {}
if 'can_evaluate_finetuned' in locals() and can_evaluate_finetuned and 'eval_model' in locals():
    models_to_eval["finetuned"] = eval_model
# if not needed comment out finetuning for base model
# if 'can_evaluate_base' in locals() and can_evaluate_base and 'base_model_eval' in locals():
#     models_to_eval["base"] = base_model_eval

results_list = []
# Check dataset and models before looping
if 'chartqa_test_dataset' in locals() and chartqa_test_dataset and models_to_eval:
    print(f"Generating predictions for {len(models_to_eval)} model(s) on {len(chartqa_test_dataset)} samples...")
    # --- Generate Predictions ---
    for sample in tqdm(chartqa_test_dataset, desc="Generating Predictions"):
        entry = {"id": sample.get("img_idx", "N/A"),
                 "question": sample.get("query"),
                 "ground_truth": str(sample.get("label"))} # Convert GT to string here

        # Basic validation of core sample data needed for processing
        if not all([entry["id"] != "N/A", entry["question"], entry["ground_truth"] is not None, isinstance(sample.get("image"), Image.Image)]):
            print(f"Warning: Skipping sample {entry['id']} due to missing/invalid core data.")
            continue # Skip this sample

        # Generate for applicable models
        for name, model_obj in models_to_eval.items():
            entry[f"predicted_answer_{name}"] = generate_prediction_concise(model_obj, eval_processor, sample, MAX_NEW_TOKENS_EVAL, MAX_SEQ_LENGTH)
        results_list.append(entry)
    # --- End Prediction Loop ---

    # --- Save results_df to JSON ---
    if results_list:
        results_df = pd.DataFrame(results_list)
        if 'EVAL_OUTPUT_FILE' in locals() and EVAL_OUTPUT_FILE:
            try:
                os.makedirs(os.path.dirname(EVAL_OUTPUT_FILE), exist_ok=True)
                results_df.to_json(EVAL_OUTPUT_FILE, orient="records", indent=2)
                print(f"\nEvaluation results (raw predictions) saved to: {EVAL_OUTPUT_FILE}")
            except Exception as e_save:
                 print(f"\nERROR saving evaluation results to {EVAL_OUTPUT_FILE}: {e_save}")
        else:
             print("\nWarning: EVAL_OUTPUT_FILE not defined, results not saved.")
    else:
        print("No valid evaluation results were generated to save.")

else:
    print("Skipping evaluation generation: Dataset or models not ready.")

print(f"\n--- Evaluation Generation & Saving Finished ---")

In [ ]:
### Final results compilation
print("--- Aggregating Evaluation Results ---")

BASE_OUTPUT_DIR = Path(WORKING_DIR)
OUTPUT_FOLDER_PREFIX = OUTPUT_DIR_BASE_NAME
EVAL_FILENAME = "chartqa_evaluation_results_comparison.json"

search_pattern = f"{OUTPUT_FOLDER_PREFIX}*/{EVAL_FILENAME}"
result_files = list(BASE_OUTPUT_DIR.glob(search_pattern))

# if not result_files:
#     raise FileNotFoundError("No evaluation result files found.")

aggregated_dfs = []
base_models_added = set()

for file_path in result_files:
    run_label = file_path.parent.name.replace(f"{OUTPUT_FOLDER_PREFIX}-", "")
    parts = run_label.split('-')
    base_model_name = '-'.join(parts[:-1]) if len(parts) >= 2 else "UnknownBase"
    base_label = f"{base_model_name}-Original"

    df = pd.read_json(file_path)

    columns = ['id', 'question', 'ground_truth'] if not aggregated_dfs else ['id']
    rename_dict = {}

    if 'predicted_answer_finetuned' in df:
        columns.append('predicted_answer_finetuned')
        rename_dict['predicted_answer_finetuned'] = f"Pred_{run_label}"

    if base_model_name not in base_models_added and 'predicted_answer_base' in df:
        columns.append('predicted_answer_base')
        rename_dict['predicted_answer_base'] = f"Pred_{base_label}"
        base_models_added.add(base_model_name)

    df_subset = df[columns].rename(columns=rename_dict)
    aggregated_dfs.append(df_subset)

if aggregated_dfs:
    final_comparison_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), aggregated_dfs)
    final_comparison_df.ffill(inplace=True)
    final_comparison_df.bfill(inplace=True)
else:
    print("⚠️ No evaluation results found to aggregate.")
    final_comparison_df = pd.DataFrame()


print("Aggregated Results Preview:")
display(final_comparison_df.head())

output_file = BASE_OUTPUT_DIR / f"{OUTPUT_FOLDER_PREFIX}-ALL_RUNS_COMPARISON_WITH_BASE.csv"
final_comparison_df.to_csv(output_file, index=False)

print(f"Aggregated results saved to {output_file}")

In [ ]:
# --- Advanced accuracy metrics -------------------------------

YES, NO      = {"yes","y","true","correct"}, {"no","n","false","incorrect"}
TOL, EPS     = 0.05, 1e-9                  # ±5 %, tiny tolerance for 0
PRED_PREFIX  = "Pred_"                     # rename if your columns differ

# 2.  Helpers -------------------------------------------------------------------
def to_scalar(v):
    """Clean & convert value → float | 'yes' | 'no' | None."""
    if pd.isna(v): return None
    s = str(v).strip()
    m = re.fullmatch(r"\[['\"]?(.*?)['\"]?\]", s)
    if m: s = m.group(1)
    s = s.lower().replace(",","").replace("$","").replace("%","").strip()
    if s in YES: return "yes"
    if s in NO:  return "no"
    for fn in (float, w2n.word_to_num):
        try: return float(fn(s))
        except Exception: pass
    return None

def relaxed(gt, pred):
    return abs(gt - pred) <= (abs(gt) * TOL or EPS)

# 3.  Basic checks --------------------------------------------------------------
if not isinstance(globals().get("final_comparison_df"), pd.DataFrame):
    sys.exit("  `final_comparison_df` is missing.")

df = final_comparison_df.copy()
df["GT_proc"] = df["ground_truth"].map(to_scalar)

keep = df["GT_proc"].isin(["yes","no"]) | df["GT_proc"].apply(lambda x: isinstance(x,(int,float)))
df_filt = df[keep]
if df_filt.empty:
    sys.exit("  No yes/no/numeric ground‑truth rows after processing.")

# Metrics per run -----------------------------------------------------------
results = {}
for col in [c for c in df.columns if c.startswith(PRED_PREFIX)]:
    proc = f"{col}_proc"
    df[proc] = df[col].map(to_scalar)

    valid   = df[["GT_proc", proc]].dropna()
    numeric = valid[valid["GT_proc"].apply(lambda x:isinstance(x,(int,float))) &
                    valid[proc].apply(lambda x:isinstance(x,(int,float)))]

    em  = 100 * (valid["GT_proc"] == valid[proc]).mean() if not valid.empty else 0
    rel = 100 * numeric.apply(lambda r: relaxed(r["GT_proc"], r[proc]), axis=1).mean() if not numeric.empty else 0

    run = col.replace(PRED_PREFIX, "")
    results[run] = {"EM (%)": round(em,2),
                    "Relaxed Num (%)": round(rel,2),
                    "# Numeric": len(numeric),
                    "# Valid": len(valid)}

    print(f"{run:<15} EM={em:6.2f}%  Relaxed={rel:6.2f}%  ({len(numeric)} num / {len(valid)} valid)")

# Summary & preview ---------------------------------------------------------
summary = pd.DataFrame(results).T.sort_values("EM (%)", ascending=False)
display(summary)
display(df.head())

# Save to CSV ---------------------------------------------------------------
BASE_OUTPUT_DIR = Path(globals().get("BASE_OUTPUT_DIR", "./outputs"))
BASE_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
prefix = globals().get("OUTPUT_FOLDER_PREFIX", "RUNS")

# Save the full dataframe
out_file = BASE_OUTPUT_DIR / f"{prefix}-ALL_RUNS_COMPARISON_WITH_PROCESSED.csv"
df.to_csv(out_file, index=False)
print("💾  Saved full DataFrame:", out_file)

# Save the summary metrics dataframe
summary_file = BASE_OUTPUT_DIR / f"{prefix}-METRICS_SUMMARY.csv"
summary.to_csv(summary_file, index=True)
print("💾  Saved metrics summary:", summary_file)

In [ ]:
# Prepare and clean the summary DataFrame
def clean_summary(summary):
    df = summary.copy()
    df.reset_index(inplace=True)
    df.rename(columns={"index": "Model"}, inplace=True)

    df["is_original"] = df["Model"].str.contains("-Original")
    df["base_model"] = df["Model"].apply(lambda x: re.sub(r"-(qlora|lora|full)(-alpha-\d+)?-Original$", "-Original", x))

    df_filtered = df[~df["is_original"] | ~df.duplicated("base_model")].copy()
    df_filtered.loc[df_filtered["is_original"], "Model"] = df_filtered.loc[df_filtered["is_original"], "base_model"]

    df_filtered.drop_duplicates(subset=["Model"], inplace=True)

    df_filtered.drop(columns=["is_original", "base_model"], inplace=True)

    df_filtered.sort_values(by="Model", inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    df_filtered["Average Accuracy (%)"] = (df_filtered["EM (%)"] + df_filtered["Relaxed Num (%)"]) / 2
    return df_filtered

# Compare tuned models to originals and include originals in output
def compute_improvements(df_filtered):
    true_originals = df_filtered[
        df_filtered["Model"].str.endswith("-Original") &
        ~df_filtered["Model"].str.contains("-(lora|qlora|full)", regex=True)
    ].copy()

    tuneds = df_filtered[~df_filtered["Model"].str.endswith("-Original")].copy()

    def normalize_model_name(name):
        name = re.sub(r"-(qlora|lora|full)(-alpha(-\d+)?)*-tuned.*", "", name)
        name = re.sub(r"-Original$", "", name)
        return name

    tuneds["base_model"] = tuneds["Model"].apply(normalize_model_name)
    true_originals["base_model"] = true_originals["Model"].apply(normalize_model_name)

    originals_deduped = true_originals.drop_duplicates(subset="base_model", keep="first")

    comparison = pd.merge(
        tuneds,
        originals_deduped[["base_model", "Average Accuracy (%)"]],
        on="base_model",
        suffixes=("", "_Original"),
        how="left"
    )
    # compute Improvement score
    comparison["Improvement (%)"] = np.where(
        comparison["Average Accuracy (%)_Original"] == 0,
        comparison["Average Accuracy (%)"] - comparison["Average Accuracy (%)_Original"],
        ((comparison["Average Accuracy (%)"] / comparison["Average Accuracy (%)_Original"]) - 1) * 100
    )
    comparison = comparison.drop_duplicates(subset=["Model"])

    originals_to_append = originals_deduped.copy()
    originals_to_append["Improvement (%)"] = 0.0
    originals_to_append.rename(columns={"Average Accuracy (%)": "Average Accuracy (%)_Original"}, inplace=True)
    originals_to_append["Average Accuracy (%)"] = originals_to_append["Average Accuracy (%)_Original"]

    columns = list(comparison.columns)
    originals_to_append = originals_to_append[columns]

    final_df = pd.concat([comparison, originals_to_append], ignore_index=True)
    final_df.sort_values(by="Improvement (%)", ascending=False, inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df

df_filtered = clean_summary(summary)
comparison_sorted = compute_improvements(df_filtered)
out_file = BASE_OUTPUT_DIR / f"{prefix}-final-ranking.csv"
comparison_sorted.to_csv(out_file, index=False)
display(comparison_sorted)

In [ ]:
def load_runs(base_dir, prefix, filter_mode="base"):
    search_pattern = f"{prefix}*/trainer_state.json"
    state_files = list(base_dir.glob(search_pattern))

    if not state_files:
        raise FileNotFoundError("No trainer state files found.")

    run_data = []
    max_loss = 0

    for file_path in state_files:
        folder_name = file_path.parent.name
        run_label = folder_name.replace(f"{prefix}-", "")

        if filter_mode == "base":
            # Only folders like "SmolVLM-256M-Instruct-lora-tuned" (not extra-tuned)
            if not re.fullmatch(r".*-tuned", run_label):
                continue
        elif filter_mode == "finetuned":
            # Match folders with -tuned-xxx or _tuned_yyy or anything more than "-tuned"
            if re.fullmatch(r".*-tuned", run_label):
                continue
        else:
            raise ValueError("filter_mode must be 'base' or 'finetuned'")

        # Determine method
        if "-full-" in folder_name:
            method = "Full"
        elif "-qlora-" in folder_name:
            method = "QLoRA"
        elif "-lora-" in folder_name:
            method = "LoRA"
        else:
            method = "LoRA"

        with open(file_path, "r") as f:
            log_history = json.load(f).get("log_history", [])

        df = pd.DataFrame(log_history)
        train_df = df[df["loss"].notna() & df["step"].notna()]
        val_df = df[df["eval_loss"].notna() & df["step"].notna()]

        if not train_df.empty:
            max_loss = max(max_loss, train_df["loss"].max())
        if not val_df.empty:
            max_loss = max(max_loss, val_df["eval_loss"].max())

        # Strip suffix for cleaner display
        clean_label = re.sub(r"-tuned$", "", run_label)
        run_data.append((clean_label, train_df, val_df, method))

    run_data.sort(key=lambda x: x[0])
    return run_data, max_loss


def set_scientific_plot_style():
    """Set publication-quality plot style with advanced settings"""
    # Use academic-focused style
    plt.style.use('seaborn-v0_8-paper')

    # Set color palette - scientific publication friendly
    colors = sns.color_palette("viridis", 8)

    # Configure matplotlib with better defaults for scientific plots
    plt.rcParams.update({
        'font.family': 'serif',
        # Use default serif fonts available on most systems
        'font.serif': ['DejaVu Serif', 'Bitstream Vera Serif', 'serif'],
        'font.size': 10,
        'axes.labelsize': 11,
        'axes.titlesize': 12,
        'xtick.labelsize': 9,
        'ytick.labelsize': 9,
        'legend.fontsize': 9,
        'axes.linewidth': 0.8,
        'grid.linewidth': 0.6,
        'lines.linewidth': 1.5,
        'lines.markersize': 5,
        'axes.prop_cycle': cycler('color', colors),
        'axes.spines.top': False,
        'axes.spines.right': False,
        'axes.grid': True,
        'grid.alpha': 0.3,
        'figure.figsize': (8, 6),
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'savefig.bbox': 'tight',
        'savefig.pad_inches': 0.05,
    })


def get_method_colors():
    """Return consistent colors by method type"""
    return {
        "Full": "#1f77b4",  # blue
        "LoRA": "#2ca02c",  # green
        "QLoRA": "#d62728"  # red
    }


def plot_runs(run_data, output_path, title="Loss Curves", figsize=None):
    if not run_data:
        print("No runs to plot.")
        return

    # Set scientific plot style
    set_scientific_plot_style()

    # Get method-specific colors
    method_colors = get_method_colors()

    # Calculate grid layout based on number of runs
    ncols = min(6, len(run_data))
    nrows = math.ceil(len(run_data) / ncols)

    # Use figsize if provided, otherwise calculate based on grid
    if figsize is None:
        figsize = (ncols * 3.2, nrows * 2.5)

    # Create figure with more sophisticated layout using GridSpec
    fig = plt.figure(figsize=figsize, constrained_layout=True)
    gs = GridSpec(nrows, ncols, figure=fig)

    # Track y-limits to standardize across subplots
    y_min, y_max = float('inf'), float('-inf')

    # Create subplots and plot data
    axes = []
    for i, (label, train_df, val_df, method) in enumerate(run_data):
        row, col = i // ncols, i % ncols
        ax = fig.add_subplot(gs[row, col])
        axes.append(ax)

        # Color based on method
        method_color = method_colors.get(method, "#ff7f0e")  # orange as default

        # Plot with enhanced styling
        if not train_df.empty:
            ax.plot(
                train_df["step"], train_df["loss"],
                label="Train",
                color=method_color,
                linestyle="-",
                marker="o",
                markersize=3,
                markevery=max(1, len(train_df)//8),
                alpha=0.95
            )

            # Update y limits
            y_min = min(y_min, train_df["loss"].min() * 0.95)
            y_max = max(y_max, train_df["loss"].max() * 1.05)

        if not val_df.empty:
            ax.plot(
                val_df["step"], val_df["eval_loss"],
                label="Val",
                color=method_color,
                linestyle="--",
                marker="s",
                markersize=3,
                markevery=max(1, len(val_df)//8),
                alpha=0.85
            )

            # Update y limits
            y_min = min(y_min, val_df["eval_loss"].min() * 0.95)
            y_max = max(y_max, val_df["eval_loss"].max() * 1.05)

        # Format tick labels
        ax.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))

        # Add method indicators with colored backgrounds
        ax.set_title(f"{label}", fontsize=10, pad=6)

        # Add method indicator as a colored text label in top-right
        ax.text(
            0.97, 0.07, method,
            transform=ax.transAxes,
            fontsize=8,
            ha='right',
            va='center',
            bbox=dict(
                boxstyle='round,pad=0.3',
                facecolor=method_color,
                alpha=0.2,
                edgecolor=method_color,
                linewidth=0.5
            )
        )

        # Add small legend in each subplot for train/val distinction
        ax.legend(
            ['Train', 'Val'],
            loc='upper right',
            fontsize=7,
            framealpha=0.5,
            handlelength=1.0,
            borderpad=0.2
        )

        # Style the grid
        ax.grid(True, linestyle=':', alpha=0.3, color='gray')

    # Standardize y-limits across all plots if we have data
    if y_min != float('inf') and y_max != float('-inf'):
        for ax in axes:
            ax.set_ylim(y_min, y_max)

    # Hide unused axes
    for ax in axes[len(run_data):]:
        ax.axis("off")

    # Add suptitle with enhanced styling
    fig.suptitle(
        title,
        fontsize=14,
        y=0.98,
        fontweight='bold',
        bbox=dict(
            facecolor='white',
            alpha=0.8,
            edgecolor='lightgray',
            boxstyle='round,pad=0.5'
        )
    )

    # Add common axis labels
    fig.supxlabel("Training Steps", fontsize=11, y=0.02)
    fig.supylabel("Loss", fontsize=11, x=0.02)

    # Tight layout
    plt.tight_layout(rect=[0.02, 0.05, 0.98, 0.95])

    # Add watermark with timestamp/version info
    fig.text(
        0.98, 0.01, f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d')}",
        ha='right', va='bottom', fontsize=6, color='gray', alpha=0.7
    )

    # Save with high resolution
    fig.savefig(output_path, dpi=300, bbox_inches='tight')

    print(f"Enhanced plot saved to {output_path}")

    # Display in Jupyter if running in a notebook environment
    try:
        from IPython import get_ipython
        if get_ipython() is not None:
            plt.show()
    except (ImportError, NameError):
        pass

    return fig


def aggregate_and_plot_loss_curves():
    print("--- Plotting Base Runs ---")
    run_data, _ = load_runs(BASE_OUTPUT_DIR, OUTPUT_FOLDER_PREFIX, filter_mode="base")
    output_path = BASE_OUTPUT_DIR / f"{OUTPUT_FOLDER_PREFIX}_base_loss_curves.pdf"

    # Use PDF for publication quality
    fig = plot_runs(
        run_data,
        output_path,
        title="Training & Validation Losses During Fine-Tuning"
    )

    # Save additional PNG for quick viewing
    png_path = output_path.with_suffix('.png')
    fig.savefig(png_path, dpi=300)

    plt.close(fig)


def aggregate_and_plot_finetuned_loss_curves():
    print("--- Plotting Fine-tuned Runs ---")
    run_data, _ = load_runs(BASE_OUTPUT_DIR, OUTPUT_FOLDER_PREFIX, filter_mode="finetuned")
    output_path = BASE_OUTPUT_DIR / f"{OUTPUT_FOLDER_PREFIX}_finetuned_loss_curves.pdf"

    fig = plot_runs(
        run_data,
        output_path,
        title="Hyperparameter Tuning: Training & Validation Losses"
    )

    # Save additional PNG for quick viewing
    png_path = output_path.with_suffix('.png')
    fig.savefig(png_path, dpi=300)

    plt.close(fig)


# Run both
aggregate_and_plot_loss_curves()
aggregate_and_plot_finetuned_loss_curves()